**Autor:** __Tiago Dias__

# Desafio 7 TNT

### Importando bibliotecas

In [3]:
!pip install paho-mqtt

     |████████████████████████████████| 102kB 3.5MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.0-cp36-none-any.whl size=61415 sha256=4378eb4b50875c1e187de8f137a28db436c404f2967e8f6b66a19fffaf9bb5fc
  Stored in directory: /root/.cache/pip/wheels/02/94/6c/8474137cb7a5a3e001d70a22c8ff919caee69435376bccce79
Successfully built paho-mqtt


In [4]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from time import time, gmtime


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Extração de dados API

In [ ]:
import paho.mqtt.subscribe as subscribe

topics = ['tnt']
HOST = 'HOST'
PORT = 30573
USERNAME = 'USERNAME'
PASSWORD = 'PASSWORD'
KEEPALIVE = 600

msg = subscribe.simple(topics,
                       msg_count=17016,
                       hostname=HOST, 
                       port=PORT, 
                       auth={'username':USERNAME, 'password':PASSWORD})

In [ ]:
len(msg)

In [ ]:
import ast

lista_dict = []

for i in range((len(msg))):
  dict_str = msg[i].payload.decode("UTF-8")
  mydata = ast.literal_eval(dict_str)
  lista_dict.append(mydata)

In [ ]:
df = pd.DataFrame(data=lista_dict)
df.shape

In [ ]:
df.head(2)

### Exploração dos dados

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
todas_colunas = list(df.columns)
colunas_int = ['Original_473','Original_269','Zero','Maçã-Verde','Tangerina','Citrus','Açaí-Guaraná','Pêssego']

In [ ]:
df2 = df.copy()

for col in todas_colunas:
  if col in colunas_int:
    df2[col] = pd.to_numeric(df[col], downcast='integer')
  else:
    df2[col] = df[col].copy()

In [ ]:
df2.info()

In [ ]:
df2['Estação'].value_counts()

In [ ]:
df2['Movimentação'].value_counts()

In [ ]:
df2['TARGET'].value_counts()

### Pré processamento

In [ ]:
le = LabelEncoder()
df2['Estação'] = le.fit_transform(df2['Estação'])

### Modelo

In [ ]:
features = ['Estação','Original_473','Original_269','Zero','Maçã-Verde','Tangerina','Citrus','Açaí-Guaraná','Pêssego']


In [ ]:
X = df2[features]
y = df2.TARGET

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

### Exportando csv

In [ ]:
df.to_csv('datasetTNT.csv', index=False)